### Implementing Adversarial Validation for Data Drift
Description: Create and train a classifier that distinguishes between train and test datasets, using the classifier’s performance to infer data drift.

In [1]:
# write your code from here
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
#SimulateTrainandTestDatasetswithDrift
np.random.seed(42)
num_samples_train=1000
num_samples_test=1000
num_features=10
#Dataset1:TrainData(e.g.,fromaNormalDistribution)
X_train_data=np.random.normal(loc=0,scale=1,size=(num_samples_train,num_features))
#Dataset2:TestData(withadistributionshift,e.g.,differentmeanforsomefeatures)
X_test_data=np.random.normal(loc=0.5,scale=1.2,size=(num_samples_test,num_features))
#CombineDatasetsandCreateLabelsforAdversarialValidation
X_combined=np.vstack((X_train_data,X_test_data))
y_adversarial=np.hstack((np.zeros(num_samples_train),np.ones(num_samples_test)))
#SplitthecombineddatasetforAdversarialClassifierTraining
X_adv_train,X_adv_test,y_adv_train,y_adv_test=train_test_split(
X_combined,y_adversarial,test_size=0.3,random_state=42,stratify=y_adversarial)
#TrainanAdversarialClassifier
adversarial_classifier=LogisticRegression(random_state=42,solver='liblinear')
adversarial_classifier.fit(X_adv_train,y_adv_train)
#EvaluatetheClassifier'sPerformance
y_adv_pred_proba=adversarial_classifier.predict_proba(X_adv_test)[:,1]
auc_score=roc_auc_score(y_adv_test,y_adv_pred_proba)
print(f"AdversarialValidationAUCScore:{auc_score:.4f}")
#Interprettheresult
if auc_score>0.7: #Thresholdisarbitrary,butsignificantlyabove0.5indicatesdistinguishability
    print("SignificantDataDriftDetected:AUCscoreiswellabove0.5,indicatingthedatasetsaredistinguishable.")
elif auc_score<0.55: #Close to 0.5 suggests no significant drift
    print("NoSignificantDataDriftDetected:AUCscoreiscloseto0.5,indicatingthedatasetsarenoteasilydistinguishable.")
else:
    print("PotentialDataDrift:AUCscoreissomewhatabove0.5,suggestingsomedecreeofdistinguishability.")

AdversarialValidationAUCScore:0.8620
SignificantDataDriftDetected:AUCscoreiswellabove0.5,indicatingthedatasetsaredistinguishable.
